In [135]:
import re
import numpy as np
import requests
from bs4 import BeautifulSoup
import pandas as pd
import lxml

In [123]:
url = "https://www.centuryrealestategroup.com/for-sale/"
url_2 = "https://www.centuryrealestategroup.com/for-rent/"

In [126]:
def house_urls(url, url_page):
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    h4_tags = soup.select("h4")
    
    links = []
    for h4 in h4_tags:
        a_tag = h4.find("a")
        if a_tag is not None:
            links.append(a_tag["href"])
            
    return links

In [131]:
def page_urls(url, page_number):
    tot_urls = []
    for page in range(1, page_number+1):
        tot_urls.extend(house_urls(url, url + "page/" + str(page) + "/"))
    return tot_urls

In [141]:
for_rent = page_urls(url = url_2, page_number=5)

In [142]:
for_rent = page_urls(url = url, page_number=5)

In [145]:
def info(house_url):
    response = requests.get(house_url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    house_dict = {}
    
    house_dict["Title"] = soup.select("h2")[0].text
    house_dict["Advert type"] = soup.select("span.property-badge")[0].text
    house_dict["Price"] = soup.select("div.property-pricing")[0].text.strip('\n')
    bedrooms = soup.select("li.main-detail-_bedrooms")

    if bedrooms:
        house_dict["Bedrooms"] = bedrooms[0].text
    else:
        house_dict["Bedrooms"] = "N/A"
    
    bathrooms = soup.select("li.main-detail-_bathrooms")
    
    if bathrooms:
        house_dict["Bathrooms"] = bathrooms[0].text
    else:
        house_dict["Bathrooms"] = "N/A"
        
    house_dict["Property Type"] = soup.select("span.property-type-badge")[0].text
    plot = soup.select("li.main-detail-_area")
    
    if plot:
        house_dict["Plot size"] = plot[0].text
    else:
        house_dict["Plot size"] = "N/A"
    
    return house_dict
    

In [121]:
def final_data(urls):
    data = []
    for link in urls:
        data.append(info(link))
    df = pd.DataFrame(data)
    return df

In [143]:
final_data(for_rent[1:5])

,Title,Advert type,Price,Bedrooms,Bathrooms,Property Type,Plot size
0,CS 180 NYAMATA For Sale Commercial Land,For Sale,"RWF 195,000,000",N/A,N/A,Commercial Land,N/A
1,CS 179 NYAMATA For Sale Residential Land,For Sale,"RWF 350,000,000",N/A,N/A,Residential Land,N/A
2,CS 177 BUSANZA For Sale Apartments,For Sale,"RWF 80,000,000",3 Bedrooms,2.5 Bathrooms,Apartments,sq m 512
3,CS 176 BUGESERA For Sale Agricultural Land,For Sale,"RWF 150,000,000",N/A,N/A,Agricultural Land,N/A
